# Advanced SQL [1] - Advanced Data Engineering Pipeline

# Data Engineering Pipeline
Data engineering is the pipeline that deals with the processing/cleaning of data before it is analysed/trained by an ML model.

1. Preprocessing
2. Storing in DB
3. Pipeline

> Pipeline Flow: Extracting > Loading > Transforming > Testing/Training > Analysis/Visualisation > Evaluation > Deployment


Machine Learning pipeline is to train a model to make predictions/visualisations.

# Goal: Build a Data Engineering/ML Pipeline From Start to Finish using Advanced SQL

## Get A Dataset of MRI Scans of Different Brain Tumors, processed by Data Engineering pipeline, train ML model to predict different types of brain tumors, build a GenAI visual interface that will connect to the ML model


> ETL process is extract, transfor and load




In [2]:
# Import necessary libraries
import os
import zipfile
import sqlite3
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torch
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
import hashlib
import random
from IPython.display import display
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm
import torch.nn.functional as F
import numpy as np
import cv2
from sklearn.model_selection import train_test_split

In [3]:
#Check to see connected to computing device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


# Extracting and Loading Data

In [5]:
#File Path
zip_file_path='brain_tumor_mri.zip'
extraction_path=""

In [6]:
#Extract Files from Zip File
with zipfile.ZipFile(zip_file_path, "r") as zip_ref:
  zip_ref.extractall(extraction_path)

  for root, dirs, files in os.walk('brain_tumor_mri/'):
    print(root, "contains", len(files), "files and", len(dirs), "dictionaries.")


  train_dir = os.path.join('brain_tumor_mri/', 'training')
  test_dir = os.path.join('brain_tumor_mri/', 'testing')

brain_tumor_mri/ contains 0 files and 2 dictionaries.
brain_tumor_mri/training contains 0 files and 4 dictionaries.
brain_tumor_mri/training/notumor contains 1595 files and 0 dictionaries.
brain_tumor_mri/training/glioma contains 1321 files and 0 dictionaries.
brain_tumor_mri/training/meningioma contains 1339 files and 0 dictionaries.
brain_tumor_mri/training/pituitary contains 1457 files and 0 dictionaries.
brain_tumor_mri/testing contains 1 files and 4 dictionaries.
brain_tumor_mri/testing/notumor contains 405 files and 0 dictionaries.
brain_tumor_mri/testing/glioma contains 300 files and 0 dictionaries.
brain_tumor_mri/testing/meningioma contains 306 files and 0 dictionaries.
brain_tumor_mri/testing/pituitary contains 300 files and 0 dictionaries.


# Transforming - Hash Mapping

In [7]:
# Data Cleaning - Removing Abnormal/Invalid Values/Duplicates to reduce biases and improve accuracy
def calculate_image_hash(image_path):
  with open(image_path, "rb") as f:
    return hashlib.md5(f.read()).hexdigest()

#Establish Connection
conn = sqlite3.connect('brain_tumor_mri.db')
c = conn.cursor()

# Resetting Tables (Remove existing table, create new table)
c.execute("DROP TABLE IF EXISTS mri_data")

#Create Table
#Metadata for Table = Table Dictionary || split -> Testing or Training Data, image_hash -> Hash Mapping Values
c.execute('''
CREATE TABLE mri_data (
  image_path TEXT PRIMARY KEY,
  label TEXT,
  split TEXT,
  image_hash TEXT
)
''')

def add_images_to_db(directory, split):
  for label in os.listdir(directory):
    label_path = os.path.join(directory, label)
    if os.path.isdir(label_path):
      for img_file in os.listdir(label_path):
        if img_file.endswith(".jpg"):
          img_path = os.path.join(label_path, img_file)
          img_hash = calculate_image_hash(img_path)
          c.execute("INSERT OR IGNORE INTO mri_data (image_path, label, split, image_hash) VALUEs (?,?,?,?)",
                    (img_path, label, split, img_hash))

add_images_to_db(train_dir, "train")
add_images_to_db(test_dir, "test")

#Initialising Data Injection
conn.commit()

# Check Data Injestion and Information of mri_data Table

In [8]:
#Total Number of Data
total_counts_df = pd.read_sql_query("SELECT label, COUNT(*) AS count FROM mri_data", conn)
display(total_counts_df)

,label,count
0,notumor,7023


In [9]:
#Total Number of Data from each Label
label_counts_df = pd.read_sql_query("SELECT label, COUNT(*) AS count FROM mri_data GROUP BY label", conn)
display(label_counts_df)

,label,count
0,glioma,1621
1,meningioma,1645
2,notumor,2000
3,pituitary,1757


In [10]:
#The number of training and testing
split_counts_df = pd.read_sql_query("SELECT split, COUNT(*) AS count FROM mri_data GROUP BY split", conn)
display(split_counts_df)

,split,count
0,test,1311
1,train,5712


In [11]:
#First 5 rows of Dataset
initial_data_df = pd.read_sql_query("SELECT * FROM mri_data LIMIT 5", conn)
display(initial_data_df)

,image_path,label,split,image_hash
0,brain_tumor_mri/training/notumor/Tr-no_0440.jpg,notumor,train,f75f2a80d34787fdcc2ee44622f21460
1,brain_tumor_mri/training/notumor/Tr-no_0722.jpg,notumor,train,72e638d7e6895d1b61ea8d6e7c879cfb
2,brain_tumor_mri/training/notumor/Tr-no_1478.jpg,notumor,train,e43cc3022533c898b30efba4a989e239
3,brain_tumor_mri/training/notumor/Tr-no_1053.jpg,notumor,train,2425479e1c8ff8a4f2f852c5e6957dad
4,brain_tumor_mri/training/notumor/Tr-no_1509.jpg,notumor,train,59c81b7ba29b994e5b855d7f2bfc3bb5


In [12]:
#Display Count of Hash with Duplicates
initial_hash_counts_df = pd.read_sql_query("SELECT image_hash, COUNT(*) AS count FROM mri_data GROUP BY image_hash HAVING count > 1", conn)
display(initial_hash_counts_df)

,image_hash,count
0,04ad12436c5b59dac34ff27b48e8dd9a,3
1,05c3caea567ffd77caf79c12654b87e5,3
2,05fe39a899414d2dab6cad25c301b32c,2
3,06c3e21e888012e7845da47b26b742e8,2
4,0abe34033d22367d0fe55445050a6f7e,2
...,...,...
189,f4b4f54022e5d07d1cfecf6868492299,3
190,f6a666f3aad6223c98e59c677b06310c,2
191,fc6ff0af19a89f3e2c2c4b1d7f5b9dc6,5
192,fdea69d8f1eaf6ad0933a4e4ba3cc4f6,2


# Cleaning Data #1 - Remove Duplicates

In [13]:
#Remove Duplicates
c.execute('''
DELETE FROM mri_data
WHERE rowid NOT IN (
  SELECT MIN(rowid)
  FROM mri_data
  GROUP BY image_hash
)
''')

conn.commit()

## See if Duplicates are Removed

In [14]:
dedup_hash_counts = pd.read_sql_query("SELECT image_hash, COUNT(*) AS count FROM mri_data GROUP BY image_hash HAVING count > 1", conn)
display(dedup_hash_counts)

,image_hash,count


# Cleaning Data #2 - Abnormal Files

In [15]:
try:
  c.execute('ALTER TABLE mri_data ADD COLUMN file_name_length INTEGER')
except:
  print("Column already exists")

c.execute('''
UPDATE mri_data
SET file_name_length = LENGTH(image_path) - LENGTH(REPLACE(image_path, '/', '')) - LENGTH(SUBSTR(image_path, 0, INSTR(image_path, '/')))
''')

conn.commit()

## File Name Length Column Information

In [17]:
file_name_length_df = pd.read_sql_query("SELECT image_path, file_name_length FROM mri_data LIMIT 5", conn)
display(file_name_length_df)

,image_path,file_name_length
0,brain_tumor_mri/training/notumor/Tr-no_0440.jpg,-12
1,brain_tumor_mri/training/notumor/Tr-no_0722.jpg,-12
2,brain_tumor_mri/training/notumor/Tr-no_1478.jpg,-12
3,brain_tumor_mri/training/notumor/Tr-no_1053.jpg,-12
4,brain_tumor_mri/training/notumor/Tr-no_1509.jpg,-12


In [19]:
#Check Number of Files with the same filename length
file_name_length_dist_df = pd.read_sql_query("SELECT file_name_length, COUNT(*) AS count FROM mri_data GROUP BY file_name_length", conn)
display(file_name_length_dist_df)

#Continue Next Lesson, state on 28/11/2024

,file_name_length,count
0,-12,6726
